In [33]:
import pandas as pd
import numpy as np
import datetime as dt
import random as rd
import math
import quasardb
import quasardb.pandas as qdbpd
import plotly.graph_objects as go

In [34]:
#some parameters to seed data
#number of trades to generate
number_of_trades=1000
#how are they distributed in time, 5 -> one trade in each bin of 5s
freq_trades=5
#seed date
date = dt.datetime (2017, 3, 14, 9, 30, 0, 0)

In [35]:
#generates number_of_trades bins of size freq_trades 
index = pd.date_range(date, periods=number_of_trades, freq=str(freq_trades)+'s')
#create randonmness for trade in each bin
jitter=np.random.randint(0, freq_trades-1,number_of_trades )
jitter_timedelta=pd.to_timedelta(jitter,unit='s')
#index for number_of_trades happening randomly in bins of size freq_trades
index= index+jitter_timedelta


In [36]:
#create using index trades with an arbitrary price in range [150,155]
trades= pd.DataFrame (
    data={'price':np.round(np.random.uniform(
            low=150.0,
            high=155.0,
            size=(number_of_trades,)),2),
        'volume':np.random.randint(10, 110,number_of_trades )
         }
    ,index=index
)

In [42]:
trades[:5]

,price,volume
2017-03-14 09:30:00,152.62,96
2017-03-14 09:30:05,152.91,73
2017-03-14 09:30:10,153.57,94
2017-03-14 09:30:16,151.32,40
2017-03-14 09:30:20,152.80,32


In [38]:
#connect to QDB and insert trades
c = quasardb.Cluster("qdb://qdb-server:2836")
qdbpd.write_dataframe(trades, c, "trades",create=True)

In [39]:
#after all this hard work we can now create bars 
#here bars of 30s
query= c.query("select first(price) , max(price), min(price)  , last( price ) , sum(volume )\
                from trades in range ( 2017-03-14 ,2017-03-15 ) group by 30s ")
query_result = query.run()


In [40]:
#convert result as a pandas dataframe
tmp = dict()
for col in query_result.tables['trades']:
    tmp[col.name] = col.data
    result_as_pandas = pd.DataFrame.from_dict(tmp, orient='columns').set_index('$timestamp')
result_as_pandas[:5]

,first(price),max(price),min(price),last(price),sum(volume)
$timestamp,,,,,
2017-03-14 09:30:00,152.62,154.62,151.32,154.62,399
2017-03-14 09:30:30,150.85,153.55,150.21,151.90,217
2017-03-14 09:31:00,152.97,154.14,151.00,151.02,310
2017-03-14 09:31:30,150.93,154.78,150.65,152.81,344
2017-03-14 09:32:00,154.49,154.49,150.06,153.43,469


In [41]:
fig = go.Figure(data=[go.Candlestick(x=result_as_pandas.index,
                open=result_as_pandas['first(price)'],
                high=result_as_pandas['max(price)'],
                low=result_as_pandas['min(price)'],
                close=result_as_pandas['last(price)'])])

fig.show()

In [22]:
#tearing down data for this sample
table = c.ts("trades")
table.remove()